# Iperf data exploration

## How data is  collected
Iperf data is collected by running iperf3 test with test server **clearskystatus.info**

**Commands**:  
Bandwidth test:
>/usr/bin/iperf3 -c clearskystatus.info

Reverse bandwidth test(server sends data to the client):
>/usr/bin/iperf3 -c clearskystatus.info -R

Data being collected(???):
 - **Ping latency** 
 - **Upload speed**
 - **Download speed**

## How data looks like

In [1]:
from data_exploration import *
import numpy as np

In [2]:
import pyodbc
def connect_to_mssql():
    with open('../credentials.json', 'r') as f_credentials:
        credentials_config = json.load(f_credentials)
    password=credentials_config['mssql_password']
    srv=credentials_config['mssql_host']
    connection = pyodbc.connect(driver='/usr/local/lib/libtdsodbc.so', server=srv,port='1433', database='net_speed_md', uid='cybera_sql', pwd=password)
    return connection

In [3]:
#Set up starting point, by default if will start from current time
starting_point=datetime.now().strftime('%Y-%m-%d %H:%M:%S')
#starting point="2019-01-10 14:00:00"  # to set upl alternative starting point
print("Starting point:",starting_point )

title_tail=" to the date "+ starting_point

Starting point: 2019-02-06 21:07:22


In [4]:
time_interval='4w' #5d

Set up influxdb connection:

In [5]:
client, client_df = connect_to_influxdb()

In [6]:
query1 = "SELECT * FROM SPEEDTEST_UPLOAD WHERE PROVIDER='iperf' ORDER BY time DESC LIMIT 10;"
df1 = get_dataframe_from_influxdb(client_df=client_df,query_influx=query1,table_name='SPEEDTEST_UPLOAD')
df1

,time,IP,PI_MAC,PROVIDER,PROVINCE,SK_PI,TEST_SERVER,UPLOAD
7,2018-12-03 08:41:47-06:00,199.116.232.249,02-99-03-82-52-06,iperf,iperf,3,clearskystatus.info,2529.28
6,2018-12-03 04:59:09-06:00,204.112.96.94,02-cd-05-c1-92-3f,iperf,iperf,4,clearskystatus.info,718.00
0,2018-12-02 10:27:57-06:00,204.112.96.66,02-07-06-80-d4-7c,iperf,iperf,7,clearskystatus.info,4311.04
1,2018-12-02 10:28:23-06:00,204.9.14.110,02-0e-04-82-02-40,iperf,iperf,9,clearskystatus.info,3133.44
2,2018-12-02 14:10:05-06:00,204.9.14.110,02-0e-04-82-02-40,iperf,iperf,9,clearskystatus.info,6522.88
3,2018-12-02 17:52:07-06:00,204.9.14.110,02-0e-04-82-02-40,iperf,iperf,9,clearskystatus.info,7444.48
4,2018-12-03 01:18:00-06:00,204.9.14.110,02-0e-04-82-02-40,iperf,iperf,9,clearskystatus.info,2150.40
8,2018-12-03 12:24:06-06:00,204.9.14.110,02-0e-04-82-02-40,iperf,iperf,9,clearskystatus.info,3205.12
5,2018-12-03 01:18:24-06:00,38.64.198.5,02-d8-06-c2-53-e7,iperf,iperf,11,clearskystatus.info,1669.12
9,2018-12-03 16:05:59-06:00,38.64.198.5,02-d8-06-c2-53-e7,iperf,iperf,11,clearskystatus.info,1443.84


Let's take just one device, for example 4:

In [7]:
query1 = "SELECT * FROM SPEEDTEST_UPLOAD WHERE PROVIDER='iperf' AND SK_PI='4' ORDER BY time DESC LIMIT 10;"
df1 = get_dataframe_from_influxdb(client_df=client_df,query_influx=query1,table_name='SPEEDTEST_UPLOAD')
df1

,time,IP,PI_MAC,PROVIDER,PROVINCE,SK_PI,TEST_SERVER,UPLOAD
0,2018-11-24 10:04:17-06:00,204.112.96.94,02-cd-05-c1-92-3f,iperf,iperf,4,clearskystatus.info,767
1,2018-11-24 13:46:18-06:00,204.112.96.94,02-cd-05-c1-92-3f,iperf,iperf,4,clearskystatus.info,775
2,2018-11-25 04:36:57-06:00,204.112.96.94,02-cd-05-c1-92-3f,iperf,iperf,4,clearskystatus.info,792
3,2018-11-25 23:04:18-06:00,204.112.96.94,02-cd-05-c1-92-3f,iperf,iperf,4,clearskystatus.info,780
4,2018-11-26 06:28:05-06:00,204.112.96.94,02-cd-05-c1-92-3f,iperf,iperf,4,clearskystatus.info,834
5,2018-11-26 10:10:15-06:00,204.112.96.94,02-cd-05-c1-92-3f,iperf,iperf,4,clearskystatus.info,768
6,2018-11-28 06:34:13-06:00,204.112.96.94,02-cd-05-c1-92-3f,iperf,iperf,4,clearskystatus.info,787
7,2018-11-28 10:16:08-06:00,204.112.96.94,02-cd-05-c1-92-3f,iperf,iperf,4,clearskystatus.info,723
8,2018-12-01 23:23:31-06:00,204.112.96.94,02-cd-05-c1-92-3f,iperf,iperf,4,clearskystatus.info,831
9,2018-12-03 04:59:09-06:00,204.112.96.94,02-cd-05-c1-92-3f,iperf,iperf,4,clearskystatus.info,718


Checking upload speed coming for the same device from speedtest:

In [8]:
query1 = "SELECT * FROM SPEEDTEST_UPLOAD WHERE PROVIDER!='iperf' AND SK_PI='4' ORDER BY time DESC LIMIT 10;"
df1 = get_dataframe_from_influxdb(client_df=client_df,query_influx=query1,table_name='SPEEDTEST_UPLOAD')
df1

,time,IP,PI_MAC,PROVIDER,PROVINCE,SK_PI,TEST_SERVER,UPLOAD
0,2018-12-03 03:06:01-06:00,204.112.96.94,02-cd-05-c1-92-3f,Bell MTS,MB),4,Shaw Communications (Winnipeg,0.63
1,2018-12-03 06:48:06-06:00,204.112.96.94,02-cd-05-c1-92-3f,Bell MTS,MB),4,Voyageur Internet (Winnipeg,0.60
2,2018-12-03 10:30:09-06:00,204.112.96.94,02-cd-05-c1-92-3f,Bell MTS,MB),4,Shaw Communications (Winnipeg,0.62
3,2018-12-04 01:18:08-06:00,204.112.96.94,02-cd-05-c1-92-3f,Bell MTS,MB),4,Voyageur Internet (Winnipeg,0.64
4,2018-12-04 05:00:11-06:00,204.112.96.94,02-cd-05-c1-92-3f,Bell MTS,MB),4,Voyageur Internet (Winnipeg,0.64
5,2018-12-04 12:24:11-06:00,204.112.96.94,02-cd-05-c1-92-3f,Bell MTS,MB),4,Shaw Communications (Winnipeg,0.55
6,2018-12-04 16:06:13-06:00,204.112.96.94,02-cd-05-c1-92-3f,Bell MTS,MB),4,Voyageur Internet (Winnipeg,0.60
7,2018-12-04 19:48:08-06:00,204.112.96.94,02-cd-05-c1-92-3f,Bell MTS,MB),4,Voyageur Internet (Winnipeg,0.67
8,2018-12-05 06:54:17-06:00,204.112.96.94,02-cd-05-c1-92-3f,Bell MTS,MB),4,Shaw Communications (Winnipeg,0.64
9,2018-12-05 10:36:12-06:00,204.112.96.94,02-cd-05-c1-92-3f,Bell MTS,MB),4,Shaw Communications (Winnipeg,0.66


Looks like iperf is collecting data in Kbps vs Speedtest is collecting data in Mbps.
Kbps can be converted to Mbps by multiplying by 0.001

Checking download speed and ping latency coming from the same device:

In [9]:
query2 = "SELECT * FROM SPEEDTEST_DOWNLOAD WHERE PROVIDER='iperf' AND SK_PI='4' ORDER BY time DESC LIMIT 10;"
df2 = get_dataframe_from_influxdb(client_df=client_df,query_influx=query2,table_name='SPEEDTEST_DOWNLOAD')
df2

,time,DOWNLOAD,IP,PI_MAC,PROVIDER,PROVINCE,SK_PI,TEST_SERVER
0,2018-11-24 10:04:17-06:00,3758.08,204.112.96.94,02-cd-05-c1-92-3f,iperf,iperf,4,clearskystatus.info
1,2018-11-24 13:46:18-06:00,3860.48,204.112.96.94,02-cd-05-c1-92-3f,iperf,iperf,4,clearskystatus.info
2,2018-11-25 04:36:57-06:00,3860.48,204.112.96.94,02-cd-05-c1-92-3f,iperf,iperf,4,clearskystatus.info
3,2018-11-25 23:04:18-06:00,3973.12,204.112.96.94,02-cd-05-c1-92-3f,iperf,iperf,4,clearskystatus.info
4,2018-11-26 06:28:05-06:00,3860.48,204.112.96.94,02-cd-05-c1-92-3f,iperf,iperf,4,clearskystatus.info
5,2018-11-26 10:10:15-06:00,3860.48,204.112.96.94,02-cd-05-c1-92-3f,iperf,iperf,4,clearskystatus.info
6,2018-11-28 06:34:13-06:00,3860.48,204.112.96.94,02-cd-05-c1-92-3f,iperf,iperf,4,clearskystatus.info
7,2018-11-28 10:16:08-06:00,3655.68,204.112.96.94,02-cd-05-c1-92-3f,iperf,iperf,4,clearskystatus.info
8,2018-12-01 23:23:31-06:00,3860.48,204.112.96.94,02-cd-05-c1-92-3f,iperf,iperf,4,clearskystatus.info
9,2018-12-03 04:59:09-06:00,3758.08,204.112.96.94,02-cd-05-c1-92-3f,iperf,iperf,4,clearskystatus.info


In [10]:
query3 = "SELECT * FROM SPEEDTEST_PING WHERE PROVIDER='iperf' AND SK_PI='4' ORDER BY time DESC LIMIT 10;"
df3 = get_dataframe_from_influxdb(client_df=client_df,query_influx=query3,table_name='SPEEDTEST_PING')
df3

,time,IP,PING,PI_MAC,PROVIDER,PROVINCE,SK_PI,TEST_SERVER
0,2018-11-24 10:04:17-06:00,204.112.96.94,57.105,02-cd-05-c1-92-3f,iperf,iperf,4,clearskystatus.info
1,2018-11-24 13:46:18-06:00,204.112.96.94,58.740,02-cd-05-c1-92-3f,iperf,iperf,4,clearskystatus.info
2,2018-11-25 04:36:57-06:00,204.112.96.94,56.491,02-cd-05-c1-92-3f,iperf,iperf,4,clearskystatus.info
3,2018-11-25 23:04:18-06:00,204.112.96.94,59.371,02-cd-05-c1-92-3f,iperf,iperf,4,clearskystatus.info
4,2018-11-26 06:28:05-06:00,204.112.96.94,55.602,02-cd-05-c1-92-3f,iperf,iperf,4,clearskystatus.info
5,2018-11-26 10:10:15-06:00,204.112.96.94,56.270,02-cd-05-c1-92-3f,iperf,iperf,4,clearskystatus.info
6,2018-11-28 06:34:13-06:00,204.112.96.94,55.919,02-cd-05-c1-92-3f,iperf,iperf,4,clearskystatus.info
7,2018-11-28 10:16:08-06:00,204.112.96.94,56.454,02-cd-05-c1-92-3f,iperf,iperf,4,clearskystatus.info
8,2018-12-01 23:23:31-06:00,204.112.96.94,56.683,02-cd-05-c1-92-3f,iperf,iperf,4,clearskystatus.info
9,2018-12-03 04:59:09-06:00,204.112.96.94,57.326,02-cd-05-c1-92-3f,iperf,iperf,4,clearskystatus.info


Comparing with ping latency coming from speedtest:

In [11]:
query3 = "SELECT * FROM SPEEDTEST_PING WHERE PROVIDER!='iperf' AND SK_PI='4' ORDER BY time DESC LIMIT 10;"
df3 = get_dataframe_from_influxdb(client_df=client_df,query_influx=query3,table_name='SPEEDTEST_PING')
df3

,time,IP,PING,PI_MAC,PROVIDER,PROVINCE,SK_PI,TEST_SERVER
0,2018-12-03 03:06:01-06:00,204.112.96.94,66.148,02-cd-05-c1-92-3f,Bell MTS,MB),4,Shaw Communications (Winnipeg
1,2018-12-03 06:48:06-06:00,204.112.96.94,62.255,02-cd-05-c1-92-3f,Bell MTS,MB),4,Voyageur Internet (Winnipeg
2,2018-12-03 10:30:09-06:00,204.112.96.94,70.408,02-cd-05-c1-92-3f,Bell MTS,MB),4,Shaw Communications (Winnipeg
3,2018-12-04 01:18:08-06:00,204.112.96.94,64.617,02-cd-05-c1-92-3f,Bell MTS,MB),4,Voyageur Internet (Winnipeg
4,2018-12-04 05:00:11-06:00,204.112.96.94,68.327,02-cd-05-c1-92-3f,Bell MTS,MB),4,Voyageur Internet (Winnipeg
5,2018-12-04 12:24:11-06:00,204.112.96.94,69.748,02-cd-05-c1-92-3f,Bell MTS,MB),4,Shaw Communications (Winnipeg
6,2018-12-04 16:06:13-06:00,204.112.96.94,71.463,02-cd-05-c1-92-3f,Bell MTS,MB),4,Voyageur Internet (Winnipeg
7,2018-12-04 19:48:08-06:00,204.112.96.94,71.544,02-cd-05-c1-92-3f,Bell MTS,MB),4,Voyageur Internet (Winnipeg
8,2018-12-05 06:54:17-06:00,204.112.96.94,64.564,02-cd-05-c1-92-3f,Bell MTS,MB),4,Shaw Communications (Winnipeg
9,2018-12-05 10:36:12-06:00,204.112.96.94,68.371,02-cd-05-c1-92-3f,Bell MTS,MB),4,Shaw Communications (Winnipeg


Latencies are slightly different but not much, looks like units are the same - Miliseconds.

Let's compare with what we have in MS SQL database:

In [12]:
cnxn = connect_to_mssql()
sql = "SELECT TOP 10 * FROM FCT_SPEEDTEST WHERE PROVIDER='iperf' AND SK_PI='4' ORDER BY DATA_DATE DESC;"
pd.read_sql(sql,cnxn)

,SK_PI,SK_FILE_PATTERN,DATA_DATE,PROVIDER,IP,TEST_SERVER,PROVINCE,PING,DOWNLOAD,UPLOAD
0,4,20,2018-12-03 10:59:09.750,iperf,204.112.96.94,clearskystatus.info,iperf,57.326,3758.08,718.0
1,4,20,2018-12-02 05:23:31.210,iperf,204.112.96.94,clearskystatus.info,iperf,56.683,3860.48,831.0
2,4,20,2018-11-28 16:16:08.767,iperf,204.112.96.94,clearskystatus.info,iperf,56.454,3655.68,723.0
3,4,20,2018-11-28 12:34:13.453,iperf,204.112.96.94,clearskystatus.info,iperf,55.919,3860.48,787.0
4,4,20,2018-11-26 16:10:15.617,iperf,204.112.96.94,clearskystatus.info,iperf,56.270,3860.48,768.0
5,4,20,2018-11-26 12:28:05.287,iperf,204.112.96.94,clearskystatus.info,iperf,55.602,3860.48,834.0
6,4,20,2018-11-26 05:04:18.657,iperf,204.112.96.94,clearskystatus.info,iperf,59.371,3973.12,780.0
7,4,20,2018-11-25 10:36:57.217,iperf,204.112.96.94,clearskystatus.info,iperf,56.491,3860.48,792.0
8,4,20,2018-11-24 19:46:18.777,iperf,204.112.96.94,clearskystatus.info,iperf,58.740,3860.48,775.0
9,4,20,2018-11-24 16:04:17.457,iperf,204.112.96.94,clearskystatus.info,iperf,57.105,3758.08,767.0


Are there any zeros or NaNs?

In [13]:
sql = "SELECT  * FROM FCT_SPEEDTEST WHERE PROVIDER='iperf' AND (UPLOAD=0 OR DOWNLOAD=0 OR PING=0) ORDER BY DATA_DATE DESC;"
pd.read_sql(sql,cnxn)

,SK_PI,SK_FILE_PATTERN,DATA_DATE,PROVIDER,IP,TEST_SERVER,PROVINCE,PING,DOWNLOAD,UPLOAD


In [14]:
sql = "SELECT  * FROM FCT_SPEEDTEST WHERE PROVIDER='iperf' AND (UPLOAD IS NULL OR DOWNLOAD IS NULL OR PING IS NULL) ORDER BY DATA_DATE DESC;"
pd.read_sql(sql,cnxn)

,SK_PI,SK_FILE_PATTERN,DATA_DATE,PROVIDER,IP,TEST_SERVER,PROVINCE,PING,DOWNLOAD,UPLOAD


No zeros and no NaNs

## Number of datapoints per device

In [15]:
device_numbers=get_tag_values_influxdb(client_influx=client,table_name='SPEEDTEST_DOWNLOAD', tag_name='SK_PI')
device_numbers=list(map(int, device_numbers))
device_numbers= sorted(device_numbers)
print(device_numbers)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 17, 18]


Getting number of data points per device for the entire period of time.

In [16]:
query_download_counts = "SELECT COUNT(DOWNLOAD) FROM SPEEDTEST_DOWNLOAD WHERE PROVIDER='iperf' AND time<= '"+starting_point+"' AND DOWNLOAD>0 GROUP BY SK_PI;"
download_counts=get_stats_influxdb(client_influx=client,
                               query_influx=query_download_counts,
                               stat_name='count',
                               device_numbers=device_numbers)

In [17]:
simple_bar_plot(xvalues=device_numbers,
                yvalues=download_counts,
                name="ping datapoints",
                title="Number of data points per device "+ title_tail,
                ytitle="Number of datapoints")

Some of the devices have small number of datapoints, may be they are just installed? Lets check how many dataponts came in last 4 weeks.

Getting number of datapoints per device in last 4 weeks.

In [18]:
query_download_counts_time = "SELECT COUNT(DOWNLOAD) FROM SPEEDTEST_DOWNLOAD WHERE time >= '"+starting_point+"'-"+time_interval+" AND PROVIDER='iperf' AND DOWNLOAD>0 GROUP BY SK_PI ;"
download_counts_time = get_stats_influxdb(client_influx=client,
                                      query_influx=query_download_counts_time,
                                      stat_name='count',
                                      device_numbers=device_numbers)

Plotting combined barchart - entire number of datapoints vs number of datapoints in last 4 weeks.

In [19]:
combined_bar_plot_2traces(xvalues=device_numbers,
                          yvalues1=download_counts_time,
                          yvalues2=[a - b for a, b in zip(download_counts, download_counts_time)],
                          name1='Last '+time_interval,
                          name2='The rest of the time',
                          title="Comparing number of datapoints in last "+time_interval+" vs entire time "+ title_tail,
                          ytitle="Number of datapoints")

There are no datapoints in the last 4 weeks. Let's check last reporting time for every device.

In [20]:
query_upload_last = "SELECT LAST(UPLOAD), time FROM SPEEDTEST_UPLOAD WHERE PROVIDER='iperf' AND time <= '"+starting_point+"' AND UPLOAD>0 GROUP BY SK_PI;"
result_upload_last=get_stats_influxdb(client_influx=client,
                               query_influx=query_upload_last,
                               stat_name='time',
                               device_numbers=device_numbers)

In [21]:
query_upload_first = "SELECT FIRST(UPLOAD), time FROM SPEEDTEST_UPLOAD WHERE PROVIDER='iperf' AND time <= '"+starting_point+"' AND UPLOAD>0 GROUP BY SK_PI;"
result_upload_first=get_stats_influxdb(client_influx=client,
                               query_influx=query_upload_first,
                               stat_name='time',
                               device_numbers=device_numbers)

In [22]:
print("Iperf reporting times:")
data=[]
for i in range(len(device_numbers)):
    try:
        result_upload_first[i] = dateutil.parser.parse(result_upload_first[i]).strftime('%Y-%m-%d %H:%M:%S')
    except:
        result_upload_first[i]=None
    try:    
        result_upload_last[i] = dateutil.parser.parse(result_upload_last[i]).strftime('%Y-%m-%d %H:%M:%S')
    except:
        result_upload_last[i]=None
    print("Device: ", device_numbers[i],"  was reporting from ", result_upload_first[i], " to ",result_upload_last[i])
    trace = go.Scatter(x=[result_upload_first[i],result_upload_last[i]],y=[device_numbers[i],device_numbers[i]], 
                       name = device_numbers[i],marker=dict(color=colors[i]))
    data.append(trace)
layout = dict(title = "Device reporting times(speedtest) "+ title_tail,xaxis=dict(title="Time"),
        yaxis=dict(title="Device Number"))
fig = go.Figure(data=data, layout=layout)
iplot(fig)

Iperf reporting times:
Device:  1   was reporting from  2018-10-10 22:38:54  to  2018-10-11 10:10:20
Device:  2   was reporting from  2018-10-11 15:58:37  to  2018-11-03 16:52:07
Device:  3   was reporting from  2018-10-15 21:28:35  to  2018-12-03 14:41:47
Device:  4   was reporting from  2018-10-27 01:56:01  to  2018-12-03 10:59:09
Device:  5   was reporting from  2018-10-26 20:10:16  to  2018-11-30 23:46:22
Device:  6   was reporting from  2018-10-26 23:53:09  to  2018-10-31 14:52:59
Device:  7   was reporting from  2018-11-11 09:52:14  to  2018-12-02 16:27:57
Device:  8   was reporting from  2018-11-07 20:14:27  to  2018-12-01 21:58:23
Device:  9   was reporting from  2018-12-01 14:34:26  to  2018-12-03 18:24:06
Device:  10   was reporting from  None  to  None
Device:  11   was reporting from  2018-12-01 07:10:18  to  2018-12-03 22:05:59
Device:  12   was reporting from  2018-12-01 10:52:23  to  2018-12-02 01:40:24
Device:  14   was reporting from  None  to  None
Device:  15   was r

Looks like iperf3 stopped listening on the test server on Dec3.  
Able to ping `clearskystatus.info` but all iperf3 test failing:
   >/usr/bin/iperf3 -c clearskystatus.info  
   >iperf3: error - unable to connect to server: Operation timed out

## How often data was collected

Lets check devices 2 and 4 and see how often data was collected:

In [23]:
query2 = "SELECT * FROM SPEEDTEST_DOWNLOAD WHERE PROVIDER='iperf' AND SK_PI='4' ORDER BY time DESC LIMIT 10;"
df2 = get_dataframe_from_influxdb(client_df=client_df,query_influx=query2,table_name='SPEEDTEST_DOWNLOAD')
df2

,time,DOWNLOAD,IP,PI_MAC,PROVIDER,PROVINCE,SK_PI,TEST_SERVER
0,2018-11-24 10:04:17-06:00,3758.08,204.112.96.94,02-cd-05-c1-92-3f,iperf,iperf,4,clearskystatus.info
1,2018-11-24 13:46:18-06:00,3860.48,204.112.96.94,02-cd-05-c1-92-3f,iperf,iperf,4,clearskystatus.info
2,2018-11-25 04:36:57-06:00,3860.48,204.112.96.94,02-cd-05-c1-92-3f,iperf,iperf,4,clearskystatus.info
3,2018-11-25 23:04:18-06:00,3973.12,204.112.96.94,02-cd-05-c1-92-3f,iperf,iperf,4,clearskystatus.info
4,2018-11-26 06:28:05-06:00,3860.48,204.112.96.94,02-cd-05-c1-92-3f,iperf,iperf,4,clearskystatus.info
5,2018-11-26 10:10:15-06:00,3860.48,204.112.96.94,02-cd-05-c1-92-3f,iperf,iperf,4,clearskystatus.info
6,2018-11-28 06:34:13-06:00,3860.48,204.112.96.94,02-cd-05-c1-92-3f,iperf,iperf,4,clearskystatus.info
7,2018-11-28 10:16:08-06:00,3655.68,204.112.96.94,02-cd-05-c1-92-3f,iperf,iperf,4,clearskystatus.info
8,2018-12-01 23:23:31-06:00,3860.48,204.112.96.94,02-cd-05-c1-92-3f,iperf,iperf,4,clearskystatus.info
9,2018-12-03 04:59:09-06:00,3758.08,204.112.96.94,02-cd-05-c1-92-3f,iperf,iperf,4,clearskystatus.info


In [24]:
query2 = "SELECT * FROM SPEEDTEST_DOWNLOAD WHERE PROVIDER='iperf' AND SK_PI='4' ORDER BY time DESC LIMIT 10;"
df2 = get_dataframe_from_influxdb(client_df=client_df,query_influx=query2,table_name='SPEEDTEST_DOWNLOAD')
df2

,time,DOWNLOAD,IP,PI_MAC,PROVIDER,PROVINCE,SK_PI,TEST_SERVER
0,2018-11-24 10:04:17-06:00,3758.08,204.112.96.94,02-cd-05-c1-92-3f,iperf,iperf,4,clearskystatus.info
1,2018-11-24 13:46:18-06:00,3860.48,204.112.96.94,02-cd-05-c1-92-3f,iperf,iperf,4,clearskystatus.info
2,2018-11-25 04:36:57-06:00,3860.48,204.112.96.94,02-cd-05-c1-92-3f,iperf,iperf,4,clearskystatus.info
3,2018-11-25 23:04:18-06:00,3973.12,204.112.96.94,02-cd-05-c1-92-3f,iperf,iperf,4,clearskystatus.info
4,2018-11-26 06:28:05-06:00,3860.48,204.112.96.94,02-cd-05-c1-92-3f,iperf,iperf,4,clearskystatus.info
5,2018-11-26 10:10:15-06:00,3860.48,204.112.96.94,02-cd-05-c1-92-3f,iperf,iperf,4,clearskystatus.info
6,2018-11-28 06:34:13-06:00,3860.48,204.112.96.94,02-cd-05-c1-92-3f,iperf,iperf,4,clearskystatus.info
7,2018-11-28 10:16:08-06:00,3655.68,204.112.96.94,02-cd-05-c1-92-3f,iperf,iperf,4,clearskystatus.info
8,2018-12-01 23:23:31-06:00,3860.48,204.112.96.94,02-cd-05-c1-92-3f,iperf,iperf,4,clearskystatus.info
9,2018-12-03 04:59:09-06:00,3758.08,204.112.96.94,02-cd-05-c1-92-3f,iperf,iperf,4,clearskystatus.info


Just by observing the data there is no consistency. 
Let's calculate the time intervals for all available datapoints for today for devices 2 and 4.

In [25]:
query_device4 = "SELECT * FROM SPEEDTEST_DOWNLOAD WHERE PROVIDER='iperf' AND SK_PI='4' AND time<= '"+starting_point+"';"
df_device4 = get_dataframe_from_influxdb(client_df=client_df,query_influx=query_device4,table_name='SPEEDTEST_DOWNLOAD')
df_device4.head()

,time,DOWNLOAD,IP,PI_MAC,PROVIDER,PROVINCE,SK_PI,TEST_SERVER
0,2018-10-26 20:56:01-05:00,3758.08,204.112.96.94,02-cd-05-c1-92-3f,iperf,iperf,4,clearskystatus.info
1,2018-10-27 02:16:18-05:00,3758.08,204.112.96.94,02-cd-05-c1-92-3f,iperf,iperf,4,clearskystatus.info
2,2018-10-27 17:04:19-05:00,3758.08,204.112.96.94,02-cd-05-c1-92-3f,iperf,iperf,4,clearskystatus.info
3,2018-10-28 11:34:03-05:00,3973.12,204.112.96.94,02-cd-05-c1-92-3f,iperf,iperf,4,clearskystatus.info
4,2018-10-29 20:52:20-05:00,3440.64,204.112.96.94,02-cd-05-c1-92-3f,iperf,iperf,4,clearskystatus.info


In [26]:
df_device4['interval'] = df_device4['time'] - df_device4['time'].shift(+1)
df_device4['interval']=round(df_device4['interval'].dt.total_seconds() / 60)

In [27]:
df_device4 = df_device4[np.isfinite(df_device4['interval'])]
df_device4.head()

,time,DOWNLOAD,IP,PI_MAC,PROVIDER,PROVINCE,SK_PI,TEST_SERVER,interval
1,2018-10-27 02:16:18-05:00,3758.08,204.112.96.94,02-cd-05-c1-92-3f,iperf,iperf,4,clearskystatus.info,320.0
2,2018-10-27 17:04:19-05:00,3758.08,204.112.96.94,02-cd-05-c1-92-3f,iperf,iperf,4,clearskystatus.info,888.0
3,2018-10-28 11:34:03-05:00,3973.12,204.112.96.94,02-cd-05-c1-92-3f,iperf,iperf,4,clearskystatus.info,1110.0
4,2018-10-29 20:52:20-05:00,3440.64,204.112.96.94,02-cd-05-c1-92-3f,iperf,iperf,4,clearskystatus.info,1998.0
5,2018-10-30 00:34:14-05:00,3860.48,204.112.96.94,02-cd-05-c1-92-3f,iperf,iperf,4,clearskystatus.info,222.0


In [28]:
time_intervals=df_device4['interval'].unique()
time_intervals= sorted(time_intervals)
print("Time intervals for device 4: ",time_intervals)

Time intervals for device 4:  [222.0, 320.0, 444.0, 445.0, 666.0, 888.0, 891.0, 1107.0, 1109.0, 1110.0, 1554.0, 1776.0, 1998.0, 2442.0, 2664.0, 3108.0, 5107.0]


In [29]:
print("Frequencies for every time interval for device4:")
df_device4.groupby(['interval']).size()

Frequencies for every time interval for device4:


interval
222.0     27
320.0      1
444.0     10
445.0      1
666.0      9
888.0      5
891.0      1
1107.0     1
1109.0     1
1110.0     3
1554.0     1
1776.0     2
1998.0     1
2442.0     1
2664.0     3
3108.0     1
5107.0     1
dtype: int64

In [30]:
query_device2 = "SELECT * FROM SPEEDTEST_DOWNLOAD WHERE PROVIDER='iperf' AND SK_PI='2' AND time<= '"+starting_point+"';"
df_device2 = get_dataframe_from_influxdb(client_df=client_df,query_influx=query_device2,table_name='SPEEDTEST_DOWNLOAD')
df_device2['interval'] = df_device2['time'] - df_device2['time'].shift(+1)
df_device2['interval']=round(df_device2['interval'].dt.total_seconds() / 60)
df_device2 = df_device2[np.isfinite(df_device2['interval'])]
print("Frequencies for every time interval for device2:")
df_device2.groupby(['interval']).size()

Frequencies for every time interval for device2:


interval
211.0      1
221.0      1
222.0     76
223.0      3
233.0      1
318.0      1
443.0      1
444.0     11
665.0      1
666.0      1
667.0      2
888.0      1
1110.0     1
1332.0     1
1554.0     1
1776.0     1
dtype: int64

In [31]:
#trace=go.Histogram(x=df_device4['interval'],xbins=dict(size=222))
#fig = go.Figure(data=[trace])
#fig['layout'].update(title='Download speed histogram per device')
#iplot(fig)

In [32]:
#import plotly.figure_factory as ff
#hist_data = [df_device4['interval']]
#group_labels = ['device 4 time interval']
#fig = ff.create_distplot(hist_data, group_labels,bin_size=60)
#fig['layout']['xaxis'].update(title='Download speed (Mbps)')
#iplot(fig)

Most of data is collected with 222 mins intervals(or 444 or 666) but its not consistent

Comparing these intervals with speedtest we can see in grafana that they alternating(not happening at the same time)

![](images/grafana_ping2.png)

## Statistics by device

Since there is not a lot of data - we will select the entire database back from todays date.

In [33]:
query_download = "SELECT * FROM SPEEDTEST_DOWNLOAD WHERE PROVIDER='iperf'AND DOWNLOAD>0 AND time <'"+starting_point+"';"
download_df = get_dataframe_from_influxdb(client_df=client_df,query_influx=query_download,
                                          table_name='SPEEDTEST_DOWNLOAD')
download_df['DOWNLOAD']=download_df['DOWNLOAD']*0.001

In [34]:
query_upload = "SELECT * FROM SPEEDTEST_UPLOAD WHERE PROVIDER='iperf'AND UPLOAD>0 AND time < '"+starting_point+"';"
upload_df = get_dataframe_from_influxdb(client_df=client_df,query_influx=query_upload,
                                          table_name='SPEEDTEST_UPLOAD')
upload_df['UPLOAD']=upload_df['UPLOAD']*0.001

In [35]:
query_ping = "SELECT * FROM SPEEDTEST_PING WHERE PROVIDER='iperf'AND PING>0 AND time < '"+starting_point+"';"
ping_df = get_dataframe_from_influxdb(client_df=client_df,query_influx=query_ping,
                                          table_name='SPEEDTEST_PING')

In [36]:
download_summary=mean_max_median_min_by1(download_df,'DOWNLOAD')
device_numbers_d=download_summary["SK_PI"].unique()
download_line=go.Scatter(x=device_numbers_d,y=[50] * len(device_numbers_d), mode='markers',marker=dict(color='red'), name='50Mbps')
combined_bar_plot_4traces(xvalues=download_summary["SK_PI"],
                         yvalues1=download_summary["max"],
                         yvalues2=download_summary["mean"],
                         yvalues3=download_summary["median"],
                         yvalues4=download_summary["min"],
                         name1="Max",
                         name2="Mean",
                         name3="Median",
                         name4="Min",
                         title="Download speed by device"+ title_tail,
                         ytitle="Mbps",
                         line=download_line,
                         stack=False)

In [37]:
upload_summary=mean_max_median_min_by1(upload_df,'UPLOAD')
device_numbers_u=upload_summary["SK_PI"].unique()
upload_line=go.Scatter(x=device_numbers_u,y=[10] * len(device_numbers_u), mode='markers',marker=dict(color='red'), name='10Mbps')

combined_bar_plot_4traces(xvalues=upload_summary["SK_PI"],
                         yvalues1=upload_summary["max"],
                         yvalues2=upload_summary["mean"],
                         yvalues3=upload_summary["median"],
                         yvalues4=upload_summary["min"],
                         name1="Max",
                         name2="Mean",
                         name3="Median",
                         name4="Min",
                         title="Upload speed by device"+ title_tail,
                         ytitle="Mbps",
                         line=upload_line,
                         stack=False)

In [38]:
ping_summary=mean_max_median_min_by1(ping_df,'PING')
combined_bar_plot_4traces(xvalues=ping_summary["SK_PI"],
                         yvalues1=ping_summary["max"],
                         yvalues2=ping_summary["mean"],
                         yvalues3=ping_summary["median"],
                         yvalues4=ping_summary["min"],  
                         name1="Max",
                         name2="Mean",
                         name3="Median",
                         name4="Min",
                         title="Ping latency by device back"+ title_tail,
                         ytitle="Miliseconds",
                         stack=False)

In [39]:
simple_boxplot(dataframe=download_df,plot_value='DOWNLOAD',sort_value='SK_PI',
               title="Download speed by device "+ title_tail,
               ytitle="Mbps", downloadline=True)

In [40]:
simple_boxplot(dataframe=upload_df,plot_value='UPLOAD',sort_value='SK_PI',
               title="Upload speed by device "+ title_tail,
               ytitle="Mbps", uploadline=True)

In [41]:
simple_boxplot(dataframe=ping_df,plot_value='PING',sort_value='SK_PI',
               title="Ping latency by device "+ title_tail,
               ytitle="Miliseconds")

## Statistic by time of the day, day of the week

### Download speed

In [42]:
download_df["hour"]=pd.to_numeric(download_df["time"].dt.hour)

In [43]:
t="Normalized download speed by hour "+ title_tail
traces=[]
for device in device_numbers_d:
    subset=download_df[download_df["SK_PI"]==device]
    trace = go.Scatter(
        x = subset['hour'],
        y=(subset['DOWNLOAD']-subset['DOWNLOAD'].mean())/subset['DOWNLOAD'].std(),
        mode = 'markers',
        marker = dict(color=colors[device]),
        name = device
    )
    traces.append(trace)
layout = go.Layout(
        title=t,
        xaxis=dict(title="Hour of the day"),
        yaxis=dict(title="Difference to normalized speeds (Mbps)")
        )
data = traces
fig = go.Figure(data=data, layout=layout)
iplot(fig)

ValueError: 
    Invalid value of type 'numpy.int64' received for the 'name' property of scatter
        Received value: 1

    The 'name' property is a string and must be specified as:
      - A string
      - A number that will be converted to a string

In [ ]:
device_number=2
subset=download_df[download_df["SK_PI"]==device_number]
t="Download speed by hour for the device " + str(device_number)+" "+ title_tail
simple_boxplot(dataframe=subset,plot_value='DOWNLOAD',sort_value='hour',
               title=t,
               xtitle="Hour of the day", downloadline=True)

In [ ]:
download_df["time_group"]=""
download_df.loc[(download_df["hour"]>23)|(download_df["hour"]<=7),"time_group"]="night 23:00-07:00"
download_df.loc[(download_df["hour"]>7)&(download_df["hour"]<=17),"time_group"]="day 7:00-17:00"
download_df.loc[(download_df["hour"]>17)&(download_df["hour"]<=23),"time_group"]="evening 17:00-23:00"

In [ ]:
#subset=download_df[download_df["SK_PI"]==device_number]
#t="Upload speed by timegroup for the device "+str(device_number)+title_tail
#simple_boxplot(dataframe=subset,plot_value='DOWNLOAD',sort_value='time_group',
#               title=t,
 #              ytitle="Mbps",downloadline=True, jitter=True)

In [ ]:
download_df["weekday"]=download_df["time"].dt.weekday_name
download_df["weekday"] = pd.Categorical(download_df["weekday"], ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"])

In [ ]:
device_number=2
subset=download_df[download_df["SK_PI"]==device_number]
t="Download speed by day of the week"+title_tail
simple_boxplot(dataframe=download_df,plot_value='DOWNLOAD',sort_value='weekday',
               title=t,
               ytitle="Mbps", weekdays=True,jitter=True)

In [ ]:
download_df["day_group"]="Weekday"
download_df.loc[(download_df["weekday"]=="Sunday")|(download_df["weekday"]=="Saturday"),"day_group"]="Weekend"

In [ ]:
t="Download speed by day group"+title_tail
simple_boxplot(dataframe=download_df,plot_value='DOWNLOAD',sort_value='day_group',
               title=t,
               ytitle="Mbps", jitter=True, downloadline=True)

### Upload speed

In [ ]:
upload_df["hour"]=pd.to_numeric(upload_df["time"].dt.hour)

In [ ]:
traces=[]
t="Normalized upload speed by hour "+title_tail
traces=[]
for device in device_numbers_u:
    subset=upload_df[upload_df["SK_PI"]==device]
    trace = go.Scatter(
        x = subset['hour'],
        y=(subset['UPLOAD']-subset['UPLOAD'].mean())/subset['UPLOAD'].std(),
        mode = 'markers',
        marker = dict(color=colors[device]),
        name = device
    )
    traces.append(trace)
layout = go.Layout(
        title=t,
        xaxis=dict(title="Hour of the day"),
        yaxis=dict(title="Difference to normalized speeds (Mbps)")
        )
data = traces
fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [ ]:
upload_df["hour"]=pd.to_numeric(upload_df["time"].dt.hour)

device_number=3
subset=upload_df[upload_df["SK_PI"]==device_number]
t="Upload speed by hour for the device " + str(device_number)+" "+title_tail
simple_boxplot(dataframe=subset,plot_value='UPLOAD',sort_value='hour',
               title=t,
               ytitle="Upload speed (Mbps)",
               xtitle="Hour of the day", uploadline=True)

In [ ]:
upload_df["time_group"]=""
upload_df.loc[(upload_df["hour"]>23)|(upload_df["hour"]<=7),"time_group"]="night 23:00-07:00"
upload_df.loc[(upload_df["hour"]>7)&(upload_df["hour"]<=17),"time_group"]="day 7:00-17:00"
upload_df.loc[(upload_df["hour"]>17)&(upload_df["hour"]<=23),"time_group"]="evening 17:00-23:00"

In [ ]:
subset=upload_df[upload_df["SK_PI"]==device_number]
t="Upload speed by timegroup for the device "+str(device_number)+title_tail
simple_boxplot(dataframe=subset,plot_value='UPLOAD',sort_value='time_group',
               title=t,
               ytitle="Mbps",uploadline=True, jitter=True)

In [ ]:
upload_df["weekday"]=upload_df["time"].dt.weekday_name
upload_df["weekday"] = pd.Categorical(upload_df["weekday"], ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"])

In [ ]:
subset=upload_df[upload_df["SK_PI"]==device_number]
t="Upload speed by day of the week for the device "+str(device_number)+title_tail
simple_boxplot(dataframe=subset,plot_value='UPLOAD',sort_value='weekday',
               title=t,
               ytitle="Mbps",uploadline=True, weekdays=True, jitter=True)

### Ping latency

In [ ]:
ping_df["hour"]=pd.to_numeric(ping_df["time"].dt.hour)
device_numbers_p=ping_df["SK_PI"].unique()

In [ ]:
traces=[]
t="Normalized ping latency by hour "+title_tail
traces=[]
for device in device_numbers_p:
    subset=ping_df[ping_df["SK_PI"]==device]
    trace = go.Scatter(
        x = subset['hour'],
        y=(subset['PING']-subset['PING'].mean())/subset['PING'].std(),
        mode = 'markers',
        marker = dict(color=colors[device]),
        name = device
    )
    traces.append(trace)
layout = go.Layout(
        title=t,
        xaxis=dict(title="Hour of the day"),
        yaxis=dict(title="Difference to normalized latencies (Miliseconda)")
        )
data = traces
fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [ ]:
t="Ping latency by hour"+title_tail
simple_boxplot(dataframe=ping_df,plot_value='PING',sort_value='hour',
               title=t,
               ytitle="Miliseconds",
               xtitle="Hour of the day")

In [ ]:
device_number=7
by_hour_by_device_p2=mean_max_median_by2(input_dataframe=ping_df,value1="PING", value2="PING",
                                          value3="PING",group_by_value="hour", rename_columns=True)
subset=by_hour_by_device_p2[by_hour_by_device_p2["SK_PI"]==device_number]
t="Ping latency(speedtest) by hour for the device "+str(device_number)+title_tail
combined_bar_plot_3traces(xvalues=subset["hour"],
                         yvalues1=subset["max"],
                         yvalues2=subset["mean"],
                         yvalues3=subset["median"],
                         name1="Max",
                         name2="Mean",
                         name3="Median",
                         title=t,
                         xtitle="hour",
                         stack=False)

In [ ]:
ping_df["weekday"]=ping_df["time"].dt.weekday_name
ping_df["weekday"] = pd.Categorical(ping_df["weekday"], ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"])

In [ ]:
t="Ping latency(collectd) by day of the week"+title_tail
simple_boxplot(dataframe=ping_df,plot_value='PING',sort_value='weekday',
               title=t,
               ytitle="Miliseconds", weekdays=True)

In [ ]:
device_number=7
by_hour_by_device_p2=mean_max_median_by2(input_dataframe=ping_df,value1="PING", value2="PING",
                                          value3="PING",group_by_value="weekday", rename_columns=True)
subset=by_hour_by_device_p2[by_hour_by_device_p2["SK_PI"]==device_number]
t="Ping latency(speedtest) by hour for the device "+str(device_number)+title_tail
combined_bar_plot_3traces(xvalues=subset["weekday"],
                         yvalues1=subset["max"],
                         yvalues2=subset["mean"],
                         yvalues3=subset["median"],
                         name1="Max",
                         name2="Mean",
                         name3="Median",
                         title=t,
                         xtitle="weekday",
                         stack=False)